In [1]:
from flask import Flask, jsonify
import threading

app = Flask(__name__)

@app.route('/api/greet/<name>', methods=['GET'])
def greet(name):
    return jsonify(message=f"Hello, {name}!")

# To get it to work in Colab, use this line instead of the usual app.run
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [2]:
# When using the threading line above in Colab, you can now call your API in colab too!
import requests
req = requests.get("http://172.28.0.12:5000/api/greet/laurence")
print(req.status_code)
print(req.text)

INFO:werkzeug:172.28.0.12 - - [20/Jul/2024 05:19:03] "GET /api/greet/laurence HTTP/1.1" 200 -


200
{"message":"Hello, laurence!"}



In [ ]:
import unittest
from flask.testing import FlaskClient

class FlaskTestClient(FlaskClient):
    def open(self, *args, **kwargs):
        kwargs.setdefault('follow_redirects', True)
        return super().open(*args, **kwargs)

app.test_client_class = FlaskTestClient

class TestFlaskAPI(unittest.TestCase):
    def setUp(self):
        self.app = app.test_client()
        self.app.testing = True

    def test_basic_greeting(self):
        response = self.app.get('/api/greet/John')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, John!"})

    def test_url_encoding(self):
        response = self.app.get('/api/greet/John%20Doe')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, John Doe!"})

    def test_special_characters(self):
        response = self.app.get('/api/greet/@!$*')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, @!$*!"})

    def test_empty_name(self):
        response = self.app.get('/api/greet/')
        self.assertEqual(response.status_code, 404)

    def test_numeric_name(self):
        response = self.app.get('/api/greet/12345')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": "Hello, 12345!"})

    def test_long_name(self):
        long_name = "a" * 1000
        response = self.app.get(f'/api/greet/{long_name}')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": f"Hello, {long_name}!"})

    def test_html_injection(self):
        html_name = "<script>alert('test')</script>"
        response = self.app.get(f'/api/greet/{html_name}')
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json, {"message": f"Hello, {html_name}!"})

    def test_json_response_format(self):
        response = self.app.get('/api/greet/John')
        self.assertEqual(response.headers['Content-Type'], 'application/json')
        self.assertTrue(response.json)

if __name__ == '__main__':
    unittest.main(argv=[''], verbosity=2, exit=False)

test_basic_greeting (__main__.TestFlaskAPI) ... ok
test_empty_name (__main__.TestFlaskAPI) ... ok
test_html_injection (__main__.TestFlaskAPI) ... FAIL
test_json_response_format (__main__.TestFlaskAPI) ... ok
test_long_name (__main__.TestFlaskAPI) ... ok
test_numeric_name (__main__.TestFlaskAPI) ... ok
test_special_characters (__main__.TestFlaskAPI) ... ok
test_url_encoding (__main__.TestFlaskAPI) ... ok

FAIL: test_html_injection (__main__.TestFlaskAPI)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-3-a1536fef67b1>", line 49, in test_html_injection
    self.assertEqual(response.status_code, 200)
AssertionError: 404 != 200

----------------------------------------------------------------------
Ran 8 tests in 0.074s

FAILED (failures=1)
